In [1]:
import numpy as np
from PIL import Image
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, UpSampling2D, InputLayer
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import LearningRateScheduler

# Load and preprocess the image
image = Image.open('input.jpg').convert('RGB')
image_array = np.array(image)

# Crop to multiples of 64
h, w, _ = image_array.shape
h_crop = h - (h % 64)
w_crop = w - (w % 64)
image_array = image_array[:h_crop, :w_crop, :]

# Split original image into 64x64 blocks
block_size = 64
num_blocks_h = h_crop // block_size
num_blocks_w = w_crop // block_size

original_blocks = []
for i in range(num_blocks_h):
    for j in range(num_blocks_w):
        block = image_array[i*block_size:(i+1)*block_size, j*block_size:(j+1)*block_size, :]
        original_blocks.append(block)
original_blocks = np.array(original_blocks)

# Create compressed blocks (8x8)
compressed_blocks = []
for block in original_blocks:
    compressed_block = block.reshape(8, 8, 8, 8, 3).mean(axis=(2, 3))
    compressed_blocks.append(compressed_block)
compressed_blocks = np.array(compressed_blocks)

# Create compressed image
compressed_image = np.zeros((num_blocks_h*8, num_blocks_w*8, 3), dtype=np.uint8)
for i in range(num_blocks_h):
    for j in range(num_blocks_w):
        compressed_image[i*8:(i+1)*8, j*8:(j+1)*8, :] = (compressed_blocks[i*num_blocks_w + j] * 255).astype(np.uint8)
Image.fromarray(compressed_image).save('compressed.png')

# Prepare training data
X = compressed_blocks / 255.0
y = original_blocks / 255.0

# Build a deeper decoder model
model = Sequential([
    InputLayer(input_shape=(8, 8, 3)),
    Conv2D(128, (3, 3), activation='relu', padding='same'),
    UpSampling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu', padding='same'),
    UpSampling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu', padding='same'),
    UpSampling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu', padding='same'),
    Conv2D(64, (3, 3), activation='relu', padding='same'),
    Conv2D(3, (3, 3), activation='sigmoid', padding='same')
])

# Learning rate decay function
def lr_decay(epoch):
    initial_lr = 0.001
    decay_factor = 0.1
    decay_epochs = 30
    lr = initial_lr * (decay_factor ** (epoch // decay_epochs))
    return lr

# Compile the model with learning rate decay
model.compile(optimizer=Adam(learning_rate=0.001), loss='mse')

# Add learning rate scheduler callback
lr_scheduler = LearningRateScheduler(lr_decay)

# Train the model
model.fit(X, y, epochs=100, batch_size=32, callbacks=[lr_scheduler])

# Reconstruct the image
y_pred = model.predict(X)
y_pred = (y_pred * 255).astype(np.uint8)

reconstructed_image = np.zeros((h_crop, w_crop, 3), dtype=np.uint8)
block_idx = 0
for i in range(num_blocks_h):
    for j in range(num_blocks_w):
        reconstructed_image[i*block_size:(i+1)*block_size, j*block_size:(j+1)*block_size, :] = y_pred[block_idx]
        block_idx += 1

Image.fromarray(reconstructed_image).save('reconstructed.png')

Epoch 1/100
15/15 [==============================] - 4s 32ms/step - loss: 0.0587 - lr: 0.0010
Epoch 2/100
15/15 [==============================] - 0s 32ms/step - loss: 0.0358 - lr: 0.0010
Epoch 3/100
15/15 [==============================] - 0s 32ms/step - loss: 0.0178 - lr: 0.0010
Epoch 4/100
15/15 [==============================] - 0s 32ms/step - loss: 0.0119 - lr: 0.0010
Epoch 5/100
15/15 [==============================] - 0s 32ms/step - loss: 0.0104 - lr: 0.0010
Epoch 6/100
15/15 [==============================] - 0s 33ms/step - loss: 0.0097 - lr: 0.0010
Epoch 7/100
15/15 [==============================] - 0s 32ms/step - loss: 0.0095 - lr: 0.0010
Epoch 8/100
15/15 [==============================] - 0s 32ms/step - loss: 0.0092 - lr: 0.0010
Epoch 9/100
15/15 [==============================] - 0s 32ms/step - loss: 0.0089 - lr: 0.0010
Epoch 10/100
15/15 [==============================] - 0s 33ms/step - loss: 0.0088 - lr: 0.0010
Epoch 11/100
15/15 [==============================] - 0s 33

In [1]:
import numpy as np
from PIL import Image
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, UpSampling2D, Concatenate, Dense, Reshape
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import LearningRateScheduler

# Load and preprocess the image
image = Image.open('input.jpg').convert('RGB')
image_array = np.array(image)

# Crop to multiples of 16
h, w, _ = image_array.shape
h_crop = h - (h % 16)
w_crop = w - (w % 16)
image_array = image_array[:h_crop, :w_crop, :]

# Split original image into 16x16 blocks
block_size = 16
num_blocks_h = h_crop // block_size
num_blocks_w = w_crop // block_size

original_blocks = []
for i in range(num_blocks_h):
    for j in range(num_blocks_w):
        block = image_array[i*block_size:(i+1)*block_size, j*block_size:(j+1)*block_size, :]
        original_blocks.append(block)
original_blocks = np.array(original_blocks)

# Create compressed blocks (2x2)
compressed_blocks = []
for block in original_blocks:
    compressed_block = block.reshape(2, 2, 8, 8, 3).mean(axis=(2, 3))  # Downscale by a factor of 8
    compressed_blocks.append(compressed_block)
compressed_blocks = np.array(compressed_blocks)

# Create block indices
block_indices = []
for i in range(num_blocks_h):
    for j in range(num_blocks_w):
        block_indices.append([i, j])
block_indices = np.array(block_indices)

# Normalize block indices
block_indices = block_indices / np.array([num_blocks_h, num_blocks_w])

# Prepare training data
X_compressed = compressed_blocks / 255.0
X_indices = block_indices
y = original_blocks / 255.0

# Build the model
compressed_input = Input(shape=(2, 2, 3))  # Input shape is now 2x2 for compressed blocks
indices_input = Input(shape=(2,))

# Flatten the compressed block and concatenate with indices
x = Conv2D(128, (3, 3), activation='relu', padding='same')(compressed_input)
x = UpSampling2D((2, 2))(x)  # Upsample to 4x4
x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
x = UpSampling2D((2, 2))(x)  # Upsample to 8x8
x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
x = UpSampling2D((2, 2))(x)  # Upsample to 16x16
x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)

# Flatten the output and concatenate with indices
x = tf.keras.layers.Flatten()(x)
x = Concatenate()([x, indices_input])

# Dense layers to process the concatenated data
x = Dense(256, activation='relu')(x)
x = Dense(16 * 16 * 3, activation='sigmoid')(x)  # Output shape is 16x16x3
x = Reshape((16, 16, 3))(x)

model = Model(inputs=[compressed_input, indices_input], outputs=x)

# Learning rate decay function
def lr_decay(epoch):
    initial_lr = 0.001
    decay_factor = 0.1
    decay_epochs = 30
    lr = initial_lr * (decay_factor ** (epoch // decay_epochs))
    return lr

# Compile the model with learning rate decay
model.compile(optimizer=Adam(learning_rate=0.001), loss='mse')

# Add learning rate scheduler callback
lr_scheduler = LearningRateScheduler(lr_decay)

# Train the model
model.fit([X_compressed, X_indices], y, epochs=100, batch_size=32, callbacks=[lr_scheduler])

# Reconstruct the image
y_pred = model.predict([X_compressed, X_indices])
y_pred = (y_pred * 255).astype(np.uint8)

reconstructed_image = np.zeros((h_crop, w_crop, 3), dtype=np.uint8)
block_idx = 0
for i in range(num_blocks_h):
    for j in range(num_blocks_w):
        reconstructed_image[i*block_size:(i+1)*block_size, j*block_size:(j+1)*block_size, :] = y_pred[block_idx]
        block_idx += 1

Image.fromarray(reconstructed_image).save('reconstructed.png')

Epoch 1/100
252/252 [==============================] - 5s 9ms/step - loss: 0.0119 - lr: 0.0010
Epoch 2/100
252/252 [==============================] - 2s 8ms/step - loss: 0.0054 - lr: 0.0010
Epoch 3/100
252/252 [==============================] - 2s 8ms/step - loss: 0.0048 - lr: 0.0010
Epoch 4/100
252/252 [==============================] - 2s 8ms/step - loss: 0.0045 - lr: 0.0010
Epoch 5/100
252/252 [==============================] - 2s 8ms/step - loss: 0.0043 - lr: 0.0010
Epoch 6/100
252/252 [==============================] - 2s 8ms/step - loss: 0.0042 - lr: 0.0010
Epoch 7/100
252/252 [==============================] - 2s 8ms/step - loss: 0.0041 - lr: 0.0010
Epoch 8/100
252/252 [==============================] - 2s 8ms/step - loss: 0.0041 - lr: 0.0010
Epoch 9/100
252/252 [==============================] - 2s 8ms/step - loss: 0.0040 - lr: 0.0010
Epoch 10/100
252/252 [==============================] - 2s 8ms/step - loss: 0.0040 - lr: 0.0010
Epoch 11/100
252/252 [===========================

In [ ]:
import numpy as np
from PIL import Image
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv2D, UpSampling2D, InputLayer
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import VGG16
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.callbacks import LearningRateScheduler

# Load and preprocess the image
image = Image.open('input.jpg').convert('RGB')
image_array = np.array(image)

# Crop to multiples of 64
h, w, _ = image_array.shape
h_crop = h - (h % 64)
w_crop = w - (w % 64)
image_array = image_array[:h_crop, :w_crop, :]

# Split original image into 64x64 blocks
block_size = 64
num_blocks_h = h_crop // block_size
num_blocks_w = w_crop // block_size

original_blocks = []
for i in range(num_blocks_h):
    for j in range(num_blocks_w):
        block = image_array[i*block_size:(i+1)*block_size, j*block_size:(j+1)*block_size, :]
        original_blocks.append(block)
original_blocks = np.array(original_blocks)

# Create compressed blocks (8x8)
compressed_blocks = []
for block in original_blocks:
    compressed_block = block.reshape(8, 8, 8, 8, 3).mean(axis=(2, 3))
    compressed_blocks.append(compressed_block)
compressed_blocks = np.array(compressed_blocks)

# Prepare training data
X = compressed_blocks / 255.0
y = original_blocks / 255.0

# Build the decoder model
model = Sequential([
    InputLayer(input_shape=(8, 8, 3)),
    Conv2D(128, (3, 3), activation='relu', padding='same'),
    UpSampling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu', padding='same'),
    UpSampling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu', padding='same'),
    UpSampling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu', padding='same'),
    Conv2D(64, (3, 3), activation='relu', padding='same'),
    Conv2D(3, (3, 3), activation='sigmoid', padding='same')
])

# Load VGG16 model for perceptual loss
vgg = VGG16(include_top=False, weights='imagenet', input_shape=(64, 64, 3))
vgg.trainable = False
feature_extractor = Model(inputs=vgg.input, outputs=vgg.get_layer('block3_conv3').output)

# Define VGG feature loss
def vgg_feature_loss(y_true, y_pred):
    y_true_features = feature_extractor(y_true)
    y_pred_features = feature_extractor(y_pred)
    return MeanSquaredError()(y_true_features, y_pred_features)

# Learning rate decay function
def lr_decay(epoch):
    initial_lr = 0.01
    decay_factor = 0.5
    decay_epochs = 50
    lr = initial_lr * (decay_factor ** (epoch // decay_epochs))
    return lr

# Compile the model with VGG feature loss
model.compile(optimizer=Adam(learning_rate=0.001), loss=vgg_feature_loss)

# Train the model
lr_scheduler = LearningRateScheduler(lr_decay)
model.fit(X, y, epochs=300, batch_size=32, callbacks=[lr_scheduler])

# Reconstruct the image
y_pred = model.predict(X)
y_pred = (y_pred * 255).astype(np.uint8)

reconstructed_image = np.zeros((h_crop, w_crop, 3), dtype=np.uint8)
block_idx = 0
for i in range(num_blocks_h):
    for j in range(num_blocks_w):
        reconstructed_image[i*block_size:(i+1)*block_size, j*block_size:(j+1)*block_size, :] = y_pred[block_idx]
        block_idx += 1

Image.fromarray(reconstructed_image).save('reconstructed.png')


58889256/58889256 [==============================] - 102s 2us/step
Epoch 1/100
15/15 [==============================] - 8s 70ms/step - loss: 27.5764 - lr: 0.0010
Epoch 2/100
15/15 [==============================] - 1s 70ms/step - loss: 24.8279 - lr: 0.0010
Epoch 3/100
15/15 [==============================] - 1s 69ms/step - loss: 23.2408 - lr: 0.0010
Epoch 4/100
15/15 [==============================] - 1s 69ms/step - loss: 22.5618 - lr: 0.0010
Epoch 5/100
15/15 [==============================] - 1s 70ms/step - loss: 21.9425 - lr: 0.0010
Epoch 6/100
15/15 [==============================] - 1s 70ms/step - loss: 21.5009 - lr: 0.0010
Epoch 7/100
15/15 [==============================] - 1s 70ms/step - loss: 21.2653 - lr: 0.0010
Epoch 8/100
15/15 [==============================] - 1s 71ms/step - loss: 20.9795 - lr: 0.0010
Epoch 9/100
15/15 [==============================] - 1s 70ms/step - loss: 20.8117 - lr: 0.0010
Epoch 10/100
15/15 [==============================] - 1s 70ms/step - loss: 20.